In [6]:
with open("gh_access_token", "r") as file:
    access_token = file.read().strip()

import requests
import json
import csv

In [46]:
query = """
query($cursor: String) {
  rateLimit {
    remaining
    cost
    used
  }
  search(query:"stars:>100 language:Python" type: REPOSITORY, first: 100, after:$cursor) {
		pageInfo {
      hasNextPage
      endCursor
    }
    edges {
      node {
        ... on Repository {
          nameWithOwner
          name
          owner {
            login
          }
          url
          description
          stargazerCount
          primaryLanguage {
            name
          }
          updatedAt
          createdAt
          issues {
            totalCount
          }
          forkCount
          watchers {
            totalCount
          }
          discussions {
            totalCount
          }
        }
      }
    }
  }
}
"""

In [49]:
with open("repos.csv", "w") as file:
  writer = csv.writer(file, lineterminator="\n")
  
  row = ["name", "url", "description",
         "stars", "primaryLanguage", "updatedAt",
         "createdAt", "issuesCount", "forkCount",
         "watchersCount", "discussionsCount"
  ]
  writer.writerow(row)

In [50]:
url = "https://api.github.com/graphql"
headers = {"Authorization": f"Bearer {access_token}"}

# cursor = "Y3Vyc29yOjkwMA=="
cursor = None

count = 0



while (True):
  json = {"query": query, "variables": {"cursor": cursor}}
  response = requests.post(url, json=json, headers=headers).json()["data"]
  
  rateLimit = response["rateLimit"]
  hasNextPage = response["search"]["pageInfo"]["hasNextPage"]
  cursor = response["search"]["pageInfo"]["endCursor"]

  repos = response["search"]["edges"]

  with open("repos.csv", "a") as file:
    writer = csv.writer(file, lineterminator="\n")

    for r in repos:
      repo = r["node"]
      row = [repo["name"], repo["url"], repo["description"],
             repo["stargazerCount"], repo["primaryLanguage"]["name"], repo["updatedAt"],
             repo["createdAt"], repo["issues"]["totalCount"], repo["forkCount"],
             repo["watchers"]["totalCount"], repo["discussions"]["totalCount"]
      ]
      writer.writerow(row)

  if (count % 1 == 0):
    print(f"Requests: {count}, endCursor: {cursor}, remaining: {rateLimit['remaining']}")
  
  count += 1

  if (not hasNextPage):
     print("done")
     break


Requests: 0, endCursor: Y3Vyc29yOjEwMA==, remaining: 4972
Requests: 1, endCursor: Y3Vyc29yOjIwMA==, remaining: 4971
Requests: 2, endCursor: Y3Vyc29yOjMwMA==, remaining: 4970
Requests: 3, endCursor: Y3Vyc29yOjQwMA==, remaining: 4969
Requests: 4, endCursor: Y3Vyc29yOjUwMA==, remaining: 4968
Requests: 5, endCursor: Y3Vyc29yOjYwMA==, remaining: 4967
Requests: 6, endCursor: Y3Vyc29yOjcwMA==, remaining: 4966
Requests: 7, endCursor: Y3Vyc29yOjgwMA==, remaining: 4965
Requests: 8, endCursor: Y3Vyc29yOjkwMA==, remaining: 4964
Requests: 9, endCursor: Y3Vyc29yOjEwMDA=, remaining: 4963
done
